In [2]:
import dill
dill.dump_session('HistogramPI.db')
#dill.load_session('HistogramPI.db')
from qutip import *
import numpy as np
import scipy
import cmath
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import itertools
from numpy.random import seed
from numpy.random import rand
from itertools import product
from scipy.misc import derivative
from scipy import optimize
from functools import wraps
from time import time
from matplotlib import gridspec


In [3]:
#Wrapper to time functions
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ti = time()
        result = f(*args, **kw)
        tf = time()
        t = tf-ti
        return result, t
    return wrap

In [4]:
#Make basis and get sz values
def operatorCreation(N):
    #operator definitionis
    si = qeye(2)
    sx = 0.5*sigmax()
    sy = 0.5*sigmay()
    sz = 0.5*sigmaz()

    sx_list = []
    sy_list = []
    sz_list = []
    

    for n in range(N):
        op_list = []
        for m in range(N):
            op_list.append(si)

        op_list[n] = sx
        sx_list.append(tensor(op_list))

        op_list[n] = sy
        sy_list.append(tensor(op_list))

        op_list[n] = sz
        sz_list.append(tensor(op_list))
        
        op_list[n] = si
    id = tensor(op_list)
        
    return sx_list, sy_list, sz_list,id

In [5]:
#Construct Hamiltonian
def hamiltonian(N,B,A0):
    sx_list = operatorCreation(N)[0]
    sy_list = operatorCreation(N)[1]
    sz_list = operatorCreation(N)[2]
    H = B*sz_list[0] 
    for n in range(N-1):
        H += A0*sz_list[0]*sz_list[n+1] + A0*sx_list[0]*sx_list[n+1] + A0*sy_list[0]*sy_list[n+1]
    return H

In [6]:
#Check if dengerate
def CheckDegenerate(H):
    estates = H.eigenstates()
    lowState = estates[0][0]
    secLowState = estates[0][1]
    if (abs(lowState-secLowState))<= 1e-10:
        return True
    else:
        return False

In [7]:
#Get Ground State Energy and Wavefuntion
class GroundState:
    
    def __init__(self, N, B, A0):
        self.hamiltonian = hamiltonian(N, B, A0)
     
    @timing
    def __call__(self):
        #find ground state
        H = self.hamiltonian
        groundState= H.groundstate()
        return groundState[0],groundState[1]

In [8]:
#Make basis and get sz values
def basisCreation(N):
    sz_list = operatorCreation(N)[2]
    Sbasis = []
    basisState = []
   
    for j in range(2):
        basisState.append(basis(2,j))
    b = itertools.product(basisState,repeat=N)
    basisTensor = list(b)
    #makes Sbasis the correct dimesion of Qobj
    for i in range(2**N):
        c = basisTensor[i][0]
        for j in range(N-1):
            c = tensor(c,basisTensor[i][j+1])
        Sbasis.append(c)

    
    #get sz values for basis states
    sz = np.zeros((2**N,N), dtype = complex)
    a = [[1 for j in range(N)] for i in range(2**N)]
    for i in range(2**N):
        for j in range(N):
            #matrix element <bra|Sz|ket>
            sz[i][j] = sz_list[j].matrix_element(Sbasis[i],Sbasis[i])
    return Sbasis, sz


In [9]:
#get randomized RBM parameters (between zero and 1)
def ranRBMpar(N,M):
    par = np.random.rand(2*(N+M+N*M))
    return par

In [10]:
#Function to give RBM wavefuntion
def RBM_ansatz(par,N, M):
    Sbasis = basisCreation(N)[0]
    sz = basisCreation(N)[1]
    #make parmeters complex
    num = N+M+N*M
    parC = np.vectorize(complex)(par[:num],par[num:])
    a = parC[:N]
    b = parC[N:N+M]
    W = parC[N+M:].reshape(M,N)
    expTerm = np.zeros(2**N, dtype = complex)
    coshTerm = np.zeros((M,2**N), dtype = complex)
    hidProduct = np.zeros(2**N, dtype = complex)
    psiMValues = np.zeros(2**N, dtype = complex)
    psiM = 0*Sbasis[0]

    for i in range(2**N):
        for m in range(M):
            coshTerm[m][i] = 2*np.cosh(np.dot(W[m],sz[i]) + b[m])
    hidProduct = np.prod(coshTerm, axis = 0) 
    
    for i in range(2**N):
        expTerm[i] = np.exp(np.dot(a,sz[i]))
        psiMValues[i] = expTerm[i]*hidProduct[i]
        psiM += psiMValues[i]*Sbasis[i]
    psiNorm = psiM.unit()
    return psiNorm


In [11]:
#Variation Energy Definition
def varEnergy(par,N, M, H):
    psiM = RBM_ansatz(par,N, M)
    E = expect(H,psiM)
    norm = psiM.norm()**2
    Enorm = E/norm
    return Enorm

In [12]:
#Energy Partials
def gradEnergy(par,N, M,H):
    eps = 0.001
    return optimize.approx_fprime(par,varEnergy,eps, N, M,H)

In [13]:
#Gradient Descent
class GradDescent:
    
    def __init__(self, N, B, A0):
        H = hamiltonian(N, B, A0)
        isDeg = CheckDegenerate(H)
        if (isDeg == True):
            raise ValueError("Hamiltonian has degenerate ground state")
        else:
            self.hamiltonian = H
     
    @timing 
    def __call__(self, N, M):
        par = ranRBMpar(N,M)
        H = self.hamiltonian
        min = scipy.optimize.fmin_cg(varEnergy,par,args= (N,M,H),gtol = 1e-04, full_output=True, retall = True, disp=True)
        #Ground State
        found_gs = RBM_ansatz(min[0],N, M)
        found_gs = found_gs.unit()
        #Ground State Energy
        found_gsEnergy =varEnergy(min[0], N, M,H)

        return min, found_gs, found_gsEnergy  

In [14]:
#Error Calculation
def err(found_gs,gs,found_gsEnergy,gsEnergy):
    engErr = np.abs(found_gsEnergy-gsEnergy)
    waveFunctionErr = found_gs.dag()*gs
    waveFunctionErr = 1-waveFunctionErr.norm()
    
    return engErr,waveFunctionErr


# Histogram N=2, M=1

In [15]:
#Parameter definition 
N= 2
M=1
B = np.pi
A0 = 1
M1gdResults = [] #gives ground state estimation at each iteration of gd
M1gdState = []
M1gdTime = []

M1edState = []
M1edTime = []

M1ActualEng = []
M1engErr = []
M1stateErr = []

hisIt = np.arange(50)

In [ ]:
for i in range(len(hisIt)):
    #gradient descent
    gradDescent = GradDescent(N, B, A0)
    gd = gradDescent(N, M)
    M1gdResults.append(gd)
    M1gdState.append(gd[0])
    M1gdTime.append(gd[1])
    
    
    #exact diagonalization 
    groundState = GroundState(N,B,A0)
    ed = groundState()
    M1edState.append(ed[0])
    M1edTime.append(ed[1])
    
    #Error
    gdEngTemp = gd[0][2]
    edEngTemp = ed[0][0]
    M1ActualEng.append(edEngTemp)
    gdStateTemp = gd[0][1]
    edStateTemp = ed[0][1]
    errTemp = err(gdStateTemp,edStateTemp,gdEngTemp,edEngTemp)
    M1engErr.append(errTemp[0])
    M1stateErr.append(errTemp[1])


Optimization terminated successfully.
         Current function value: -1.820742
         Iterations: 7
         Function evaluations: 288
         Gradient evaluations: 24
Optimization terminated successfully.
         Current function value: -1.898452
         Iterations: 218
         Function evaluations: 5352
         Gradient evaluations: 446
Optimization terminated successfully.
         Current function value: -1.898451
         Iterations: 202
         Function evaluations: 4908
         Gradient evaluations: 409
Optimization terminated successfully.
         Current function value: -1.820746
         Iterations: 4
         Function evaluations: 180
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: -1.898454
         Iterations: 38
         Function evaluations: 984
         Gradient evaluations: 82
Optimization terminated successfully.
         Current function value: -1.898446
         Iterations: 157
         Function ev

In [ ]:

plt.figure(constrained_layout=True)
plt.figure(figsize=(12,10))
ttl = plt.suptitle("N = 2, M = 1, B = $\pi$",size =20)
gs = gridspec.GridSpec(ncols=3, nrows=2)
ttl.set_position([.5, 0.92])

ax1 = plt.subplot(gs[0, 0])
ax1 .hist(M1stateErr, bins=50) 
ax1 .set_xlabel("$1-|<\Psi_{RBM}|\Psi_{ED}>|^2$",size = 15)
ax1 .set_ylabel("Frequency", size = 15)

ax2 = plt.subplot(gs[0, 1])
M1counts, M1bins, M1bars = ax2.hist(M1engErr, bins=50) 
ax2.set_xlabel("$\Delta E = |E_{RBM}-E_{ED}|$",size = 15)


ax3 = plt.subplot(gs[0, 2])
ax3.hist(M1gdTime, bins=50) 
ax3.set_xlabel("Runtime",size = 15)



ax4 = plt.subplot(gs[1, :])
color = 'tab:red'
ax4.set_xlabel('Run Number',size = 15)
ax4.set_ylabel('$\Delta E = |E_{RBM}-E_{ED}|$', color=color,size = 15)
ax4.scatter(hisIt, M1engErr,color=color, s=25)
ax4.tick_params(axis='y', labelcolor=color)

ax5 = ax4.twinx()  
color = 'tab:blue'
ax5.set_ylabel('Runtime', color=color,size = 15)  
ax5.scatter(hisIt, M1gdTime, color=color, s=25)
ax5.tick_params(axis='y', labelcolor=color)



In [ ]:
smallError = 0
for i in range(50):
    if M1bins[i]<0.1:
        cutOff=i
for i in range(cutOff):
    smallError += M1counts[i]
successPer = smallError
print(successPer*2)

# Histogram N=2, M=2

In [ ]:
#Parameter definition 
N= 2
M=2
B = np.pi
A0 = 1
M2gdResults = [] #gives ground state estimation at each iteration of gd
M2gdState = []
M2gdTime = []

M2edState = []
M2edTime = []

M2ActualEng = []
M2engErr = []
M2stateErr = []

hisIt = np.arange(50)

In [ ]:
for i in range(len(hisIt)):
    #gradient descent
    gradDescent = GradDescent(N, B, A0)
    gd = gradDescent(N, M)
    M2gdResults.append(gd)
    M2gdState.append(gd[0])
    M2gdTime.append(gd[1])
    
    
    #exact diagonalization 
    groundState = GroundState(N,B,A0)
    ed = groundState()
    M2edState.append(ed[0])
    M2edTime.append(ed[1])
    
    #Error
    gdEngTemp = gd[0][2]
    edEngTemp = ed[0][0]
    M2ActualEng.append(edEngTemp)
    gdStateTemp = gd[0][1]
    edStateTemp = ed[0][1]
    errTemp = err(gdStateTemp,edStateTemp,gdEngTemp,edEngTemp)
    M2engErr.append(errTemp[0])
    M2stateErr.append(errTemp[1])


In [ ]:

plt.figure(constrained_layout=True)
plt.figure(figsize=(12,10))
ttl = plt.suptitle("N = 2, M = 2, B = $\pi$",size =20)
gs = gridspec.GridSpec(ncols=3, nrows=2)
ttl.set_position([.5, 0.92])

ax1 = plt.subplot(gs[0, 0])
ax1 .hist(M2stateErr, bins=50) 
ax1 .set_xlabel("$1-|<\Psi_{RBM}|\Psi_{ED}>|^2$",size = 15)
ax1 .set_ylabel("Frequency", size = 15)

ax2 = plt.subplot(gs[0, 1])
M2counts, M2bins, M2bars = ax2.hist(M2engErr, bins=50) 
ax2.set_xlabel("$\Delta E = |E_{RBM}-E_{ED}|$",size = 15)


ax3 = plt.subplot(gs[0, 2])
ax3.hist(M2gdTime, bins=50) 
ax3.set_xlabel("Runtime",size = 15)



ax4 = plt.subplot(gs[1, :])
color = 'tab:red'
ax4.set_xlabel('Run Number',size = 15)
ax4.set_ylabel('$\Delta E = |E_{RBM}-E_{ED}|$', color=color,size = 15)
ax4.scatter(hisIt, M2engErr,color=color, s=25)
ax4.tick_params(axis='y', labelcolor=color)

ax5 = ax4.twinx()  
color = 'tab:blue'
ax5.set_ylabel('Runtime', color=color,size = 15)  
ax5.scatter(hisIt, M2gdTime, color=color, s=25)
ax5.tick_params(axis='y', labelcolor=color)



In [ ]:
smallError = 0
for i in range(50):
    if M2bins[i]<0.1:
        cutOff=i
for i in range(cutOff):
    smallError += M2counts[i]
successPer = smallError
print(successPer*2)

# Histogram N=2, M=3

In [ ]:
#Parameter definition 
N=2
M=3
B = np.pi
A0 = 1
M3gdResults = [] #gives ground state estimation at each iteration of gd
M3gdState = []
M3gdTime = []

M3edState = []
M3edTime = []

M3ActualEng = []
M3engErr = []
M3stateErr = []

hisIt = np.arange(50)

In [ ]:
for i in range(len(hisIt)):
    #gradient descent
    gradDescent = GradDescent(N, B, A0)
    gd = gradDescent(N, M)
    M3gdResults.append(gd)
    M3gdState.append(gd[0])
    M3gdTime.append(gd[1])
    
    
    #exact diagonalization 
    groundState = GroundState(N,B,A0)
    ed = groundState()
    M3edState.append(ed[0])
    M3edTime.append(ed[1])
    
    #Error
    gdEngTemp = gd[0][2]
    edEngTemp = ed[0][0]
    M3ActualEng.append(edEngTemp)
    gdStateTemp = gd[0][1]
    edStateTemp = ed[0][1]
    errTemp = err(gdStateTemp,edStateTemp,gdEngTemp,edEngTemp)
    M3engErr.append(errTemp[0])
    M3stateErr.append(errTemp[1])


In [ ]:

plt.figure(constrained_layout=True)
plt.figure(figsize=(12,10))
ttl = plt.suptitle("N = 2, M = 2, B = $\pi$",size =20)
gs = gridspec.GridSpec(ncols=3, nrows=2)
ttl.set_position([.5, 0.92])

ax1 = plt.subplot(gs[0, 0])
ax1 .hist(M3stateErr, bins=50) 
ax1 .set_xlabel("$1-|<\Psi_{RBM}|\Psi_{ED}>|^2$",size = 15)
ax1 .set_ylabel("Frequency", size = 15)

ax2 = plt.subplot(gs[0, 1])
M3counts, M3bins, M3bars = ax2.hist(M3engErr, bins=50) 
ax2.set_xlabel("$\Delta E = |E_{RBM}-E_{ED}|$",size = 15)


ax3 = plt.subplot(gs[0, 2])
ax3.hist(M3gdTime, bins=50) 
ax3.set_xlabel("Runtime",size = 15)



ax4 = plt.subplot(gs[1, :])
color = 'tab:red'
ax4.set_xlabel('Run Number',size = 15)
ax4.set_ylabel('$\Delta E = |E_{RBM}-E_{ED}|$', color=color,size = 15)
ax4.scatter(hisIt, M3engErr,color=color, s=25)
ax4.tick_params(axis='y', labelcolor=color)

ax5 = ax4.twinx()  
color = 'tab:blue'
ax5.set_ylabel('Runtime', color=color,size = 15)  
ax5.scatter(hisIt, M3gdTime, color=color, s=25)
ax5.tick_params(axis='y', labelcolor=color)



In [ ]:
smallError = 0
for i in range(50):
    if M3bins[i]<0.1:
        cutOff=i
print(cutOff)

for i in range(cutOff):
    smallError += M3counts[i]
successPer = smallError
print(successPer*2)

# Histogram N=3, M=1

In [ ]:
#Parameter definition 
N= 3
M=1
B = np.pi
A0 = 1
N3M1gdResults = [] #gives ground state estimation at each iteration of gd
N3M1gdState = []
N3M1gdTime = []

N3M1edState = []
N3M1edTime = []

N3M1ActualEng = []
N3M1engErr = []
N3M1stateErr = []

hisIt = np.arange(50)

In [ ]:
for i in range(len(hisIt)):
    #gradient descent
    gradDescent = GradDescent(N, B, A0)
    gd = gradDescent(N, M)
    N3M1gdResults.append(gd)
    N3M1gdState.append(gd[0])
    N3M1gdTime.append(gd[1])
    
    
    #exact diagonalization 
    groundState = GroundState(N,B,A0)
    ed = groundState()
    N3M1edState.append(ed[0])
    N3M1edTime.append(ed[1])
    
    #Error
    gdEngTemp = gd[0][2]
    edEngTemp = ed[0][0]
    N3M1ActualEng.append(edEngTemp)
    gdStateTemp = gd[0][1]
    edStateTemp = ed[0][1]
    errTemp = err(gdStateTemp,edStateTemp,gdEngTemp,edEngTemp)
    N3M1engErr.append(errTemp[0])
    N3M1stateErr.append(errTemp[1])


In [ ]:

plt.figure(constrained_layout=True)
plt.figure(figsize=(12,10))
ttl = plt.suptitle("N = 3, M = 1, B = $\pi$",size =20)
gs = gridspec.GridSpec(ncols=3, nrows=2)
ttl.set_position([.5, 0.92])

ax1 = plt.subplot(gs[0, 0])
ax1 .hist(N3M1stateErr, bins=50) 
ax1 .set_xlabel("$1-|<\Psi_{RBM}|\Psi_{ED}>|^2$",size = 15)
ax1 .set_ylabel("Frequency", size = 15)

ax2 = plt.subplot(gs[0, 1])
N3M1counts, N3M1bins, N3M1bars = ax2.hist(N3M1engErr, bins=50) 
ax2.set_xlabel("$\Delta E = |E_{RBM}-E_{ED}|$",size = 15)


ax3 = plt.subplot(gs[0, 2])
ax3.hist(N3M1gdTime, bins=50) 
ax3.set_xlabel("Runtime",size = 15)



ax4 = plt.subplot(gs[1, :])
color = 'tab:red'
ax4.set_xlabel('Run Number',size = 15)
ax4.set_ylabel('$\Delta E = |E_{RBM}-E_{ED}|$', color=color,size = 15)
ax4.scatter(hisIt, N3M1engErr,color=color, s=25)
ax4.tick_params(axis='y', labelcolor=color)

ax5 = ax4.twinx()  
color = 'tab:blue'
ax5.set_ylabel('Runtime', color=color,size = 15)  
ax5.scatter(hisIt, N3M1gdTime, color=color, s=25)
ax5.tick_params(axis='y', labelcolor=color)



In [ ]:
smallError = 0
for i in range(50):
    if N3M1bins[i]<0.1:
        cutOff=i
        
for i in range(cutOff):
    smallError += N3M1counts[i]
successPer = smallError
print(successPer*2)

# Histogram N=3, M=2

In [ ]:
#Parameter definition 
N= 3
M=2
B = np.pi
A0 = 1
N3M2gdResults = [] #gives ground state estimation at each iteration of gd
N3M2gdState = []
N3M2gdTime = []

N3M2edState = []
N3M2edTime = []

N3M2ActualEng = []
N3M2engErr = []
N3M2stateErr = []

hisIt = np.arange(50)

In [ ]:
for i in range(len(hisIt)):
    #gradient descent
    gradDescent = GradDescent(N, B, A0)
    gd = gradDescent(N, M)
    N3M2gdResults.append(gd)
    N3M2gdState.append(gd[0])
    N3M2gdTime.append(gd[1])
    
    
    #exact diagonalization 
    groundState = GroundState(N,B,A0)
    ed = groundState()
    N3M2edState.append(ed[0])
    N3M2edTime.append(ed[1])
    
    #Error
    gdEngTemp = gd[0][2]
    edEngTemp = ed[0][0]
    N3M2ActualEng.append(edEngTemp)
    gdStateTemp = gd[0][1]
    edStateTemp = ed[0][1]
    errTemp = err(gdStateTemp,edStateTemp,gdEngTemp,edEngTemp)
    N3M2engErr.append(errTemp[0])
    N3M2stateErr.append(errTemp[1])


In [ ]:

plt.figure(constrained_layout=True)
plt.figure(figsize=(12,10))
ttl = plt.suptitle("N = 3, M = 2, B = $\pi$",size =20)
gs = gridspec.GridSpec(ncols=3, nrows=2)
ttl.set_position([.5, 0.92])

ax1 = plt.subplot(gs[0, 0])
ax1 .hist(N3M2stateErr, bins=50) 
ax1 .set_xlabel("$1-|<\Psi_{RBM}|\Psi_{ED}>|^2$",size = 15)
ax1 .set_ylabel("Frequency", size = 15)

ax2 = plt.subplot(gs[0, 1])
N3M2counts, N3M2bins, N3M1bars = ax2.hist(N3M2engErr, bins=50) 
ax2.set_xlabel("$\Delta E = |E_{RBM}-E_{ED}|$",size = 15)


ax3 = plt.subplot(gs[0, 2])
ax3.hist(N3M2gdTime, bins=50) 
ax3.set_xlabel("Runtime",size = 15)



ax4 = plt.subplot(gs[1, :])
color = 'tab:red'
ax4.set_xlabel('Run Number',size = 15)
ax4.set_ylabel('$\Delta E = |E_{RBM}-E_{ED}|$', color=color,size = 15)
ax4.scatter(hisIt, N3M2engErr,color=color, s=25)
ax4.tick_params(axis='y', labelcolor=color)

ax5 = ax4.twinx()  
color = 'tab:blue'
ax5.set_ylabel('Runtime', color=color,size = 15)  
ax5.scatter(hisIt, N3M2gdTime, color=color, s=25)
ax5.tick_params(axis='y', labelcolor=color)



In [ ]:
smallError = 0
for i in range(50):
    if N3M2bins[i]<0.1:
        cutOff=i
        
for i in range(cutOff):
    smallError += N3M2counts[i]
successPer = smallError
print(successPer*2)

# Histogram N=3, M=3

In [ ]:
#Parameter definition 
N= 3
M=3
B = np.pi
A0 = 1
N3M3gdResults = [] #gives ground state estimation at each iteration of gd
N3M3gdState = []
N3M3gdTime = []

N3M3edState = []
N3M3edTime = []

N3M3ActualEng = []
N3M3engErr = []
N3M3stateErr = []

hisItShort = np.arange(50)

In [ ]:
for i in range(len(hisItShort)):
    #gradient descent
    gradDescent = GradDescent(N, B, A0)
    gd = gradDescent(N, M)
    N3M3gdResults.append(gd)
    N3M3gdState.append(gd[0])
    N3M3gdTime.append(gd[1])
    
    
    #exact diagonalization 
    groundState = GroundState(N,B,A0)
    ed = groundState()
    N3M3edState.append(ed[0])
    N3M3edTime.append(ed[1])
    
    #Error
    gdEngTemp = gd[0][2]
    edEngTemp = ed[0][0]
    N3M3ActualEng.append(edEngTemp)
    gdStateTemp = gd[0][1]
    edStateTemp = ed[0][1]
    errTemp = err(gdStateTemp,edStateTemp,gdEngTemp,edEngTemp)
    N3M3engErr.append(errTemp[0])
    N3M3stateErr.append(errTemp[1])


In [ ]:
plt.figure(constrained_layout=True)
plt.figure(figsize=(12,10))
ttl = plt.suptitle("N = 3, M = 3, B = $\pi$",size =20)
gs = gridspec.GridSpec(ncols=3, nrows=2)
ttl.set_position([.5, 0.92])

ax1 = plt.subplot(gs[0, 0])
ax1 .hist(N3M3stateErr, bins=50) 
ax1 .set_xlabel("$1-|<\Psi_{RBM}|\Psi_{ED}>|^2$",size = 15)
ax1 .set_ylabel("Frequency", size = 15)

ax2 = plt.subplot(gs[0, 1])
N3M3counts, N3M3bins, N3M3bars = ax2.hist(N3M3engErr, bins=50) 
ax2.set_xlabel("$\Delta E = |E_{RBM}-E_{ED}|$",size = 15)


ax3 = plt.subplot(gs[0, 2])
ax3.hist(N3M3gdTime, bins=50) 
ax3.set_xlabel("Runtime",size = 15)



ax4 = plt.subplot(gs[1, :])
color = 'tab:red'
ax4.set_xlabel('Run Number',size = 15)
ax4.set_ylabel('$\Delta E = |E_{RBM}-E_{ED}|$', color=color,size = 15)
ax4.scatter(hisItShort, N3M3engErr,color=color, s=25)
ax4.tick_params(axis='y', labelcolor=color)

ax5 = ax4.twinx()  
color = 'tab:blue'
ax5.set_ylabel('Runtime', color=color,size = 15)  
ax5.scatter(hisItShort, N3M3gdTime, color=color, s=25)
ax5.tick_params(axis='y', labelcolor=color)





In [ ]:
smallError = 0
for i in range(50):
    if N3M3bins[i]<0.1:
        cutOff=i

for i in range(cutOff):
    smallError += N3M3counts[i]
successPer = smallError
print(successPer)